# Projeto 1 - Ciência dos Dados

Nome: __Alexandre Antar__

Nome: __Gabriel de Araujo Alves__

Nome: __Renato Laffranchi Falcao__

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
!pip install emoji wordcloud
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from emoji import UNICODE_EMOJI
import emoji
import functools
import operator
import nltk



In [2]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\gabri\OneDrive - Insper - Institudo de Ensino e Pesquisa\Projetos\Projetos colaborativos\Analise de sentimentos Twitter


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [4]:
filename = 'brahma.xlsx'

In [5]:
#base de treinamento
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Treinamento,Relevancia
0,rt @sophiaecris: @marianaa2mari foi vcs crlh !...,0
1,@moraesbrunita pelo visto na mamadeira já era ...,0
2,@purosucodosurto o cara não tem um pau tem um ...,0
3,gente vcs que tão em home office a mais tempo ...,1
4,rt @luciusnaweb: eu depois de dois latões de b...,0


In [6]:
#base de teste
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevancia
0,@elissanmanoel tão costurando brahma ? hahaha ...,0
1,rt @nayaralucas11: era só uma praia e brahma t...,0
2,arrumando meu guarda roupa eis que encontro um...,0
3,desceu foi whisky misturado c brahma kkkkkkkkk...,0
4,rt @carolaguiarcm: hj eu só queria estar numa ...,0


In [7]:
#transformando variavel Relevancia em categórica
train['Relevancia'] = train['Relevancia'].astype('category')

train['Relevancia'].cat.categories = ['Relevante negativa', 'Não relevante', 'Relevante positiva']
train.head(6)

,Treinamento,Relevancia
0,rt @sophiaecris: @marianaa2mari foi vcs crlh !...,Não relevante
1,@moraesbrunita pelo visto na mamadeira já era ...,Não relevante
2,@purosucodosurto o cara não tem um pau tem um ...,Não relevante
3,gente vcs que tão em home office a mais tempo ...,Relevante positiva
4,rt @luciusnaweb: eu depois de dois latões de b...,Não relevante
5,"acho q vou beber uma hj, desde domingo sem beb...",Relevante negativa


In [8]:
#separando o data frame por relevância do tweet
nao_relevante=train[train.Relevancia=='Não relevante']
relevante_positivo=train[train.Relevancia=='Relevante positiva']
relevante_negativo=train[train.Relevancia=='Relevante negativa']
classifica=[nao_relevante,relevante_positivo,relevante_negativo]




In [9]:
#uma lista de strings para aferir a probabilidade posteriormente,não necessariamente será usada
classifica_nao_relevante=str(list(classifica[0].Treinamento.values))
classifica_positivo=str(list(classifica[1].Treinamento.values))
classifica_negativo=str(list(classifica[2].Treinamento.values))



___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto escolhido foi a cerveja Brahma, uma marca nacional e bem popular de cerveja e a classificação dos tweets foram feitos da seguinte forma:

* -1 : Relevante negativo -> Essa classificação se refere aos tweets que mecionavam o a marca de cerveja de forma negativa, seja com reclamações acerca do produto ou associando a marca à alguma experiência ruim.
* 0 : Irrelevante -> Essa classificação se refere aos tweets que mecionavam o a marca de cerveja de forma neutra, sem reclamações ou elogios ao produto ou que não se referiram à cerveja.
* 1 : Relevante positivo -> Essa classificação se refere aos tweets que mecionavam o a marca de cerveja de forma positiva, seja com elogios acerca do produto ou associando a marca à alguma experiência boa.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

# 1° etapa:Limpar tweets referentes ao treinamento 

In [10]:
#limpeza de mensagens em relação a pontuação
#separação de emojis e palavras e separação entre emojis
def cleanup_tweet(text):
    punctuation = '[\-/!.:?;,''"@]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    text_subbed = re.sub(' +', ' ', text_subbed)
    text_subbed = emoji.get_emoji_regexp().split(text_subbed)
    separate = [text.split() for text in text_subbed]
    text_subbed = functools.reduce(operator.concat, separate)
    return text_subbed


In [11]:
#remoção de outros items que não contarão para as relevâncias
def remove_outros(items):
    i=0
    while(i<len(items)):
        # Limpa '\n'
        items[i] = items[i].lower().replace('[\n]', '')
        
        # Remove 'rt' de retweets
        if items[i][0:2] == 'rt':
            items[i] = items[i][3:]
        
        # Remove links
        elif items[i][:4] == 'http':
            del items[i]
        i+=1
            
        
    return items

In [12]:
# Remove 'stopwords' (preposições, artigos etc.) dos tweets
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords.append('')

# Define função que aplica remoção
def remove_stopwords(lista):
    removidos = lista[:]     
    for palavra in lista: 
          if palavra in stopwords: 
                removidos.remove(palavra)               
    return removidos

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
#twt=cada tweet
#aplica todas as funções anteriores para limpar as frases
def limpos(classificador):
    lis=[]
    for twt in classificador.Treinamento:
        lis.append(remove_stopwords(remove_outros(cleanup_tweet(twt))))
    return lis


In [14]:
# Aplica limpeza nas séries de cada classificação
limpeza_tweets = [limpos(classifica[0]), limpos(classifica[1]),limpos(classifica[2])]

# Limpa dados de treino
limpeza = limpos(train)

# Aplica listagem das palavras dos dados de treino
for i in range(len(limpeza)):
    limpeza[i] = ', '.join(limpeza[i])

# Cria coluna com tweets já limpos
train['Treinamento_limpo'] = limpeza



train.head()



,Treinamento,Relevancia,Treinamento_limpo
0,rt @sophiaecris: @marianaa2mari foi vcs crlh !...,Não relevante,"sophiaecris, marianaa2mari, vcs, crlh, boa, br..."
1,@moraesbrunita pelo visto na mamadeira já era ...,Não relevante,"moraesbrunita, visto, mamadeira, boa, brahma, ..."
2,@purosucodosurto o cara não tem um pau tem um ...,Não relevante,"purosucodosurto, cara, pau, latão, brahma"
3,gente vcs que tão em home office a mais tempo ...,Relevante positiva,"gente, vcs, tão, home, office, tempo, ok, mand..."
4,rt @luciusnaweb: eu depois de dois latões de b...,Não relevante,"luciusnaweb, dois, latões, brahma, bem, assim,..."


### 2° ETAPA


 

Agora com os dados limpos, o objetivo é ensinar o classificador de acordo com as probabilidades de cada palavra apresentada para cada série de relevância.

In [20]:
#value_counts-refere-se ao numero de vezes da palavra na série dos tweets não relevantes
n_rel=train[train.Relevancia=='Não relevante']
n_rel1=n_rel.Treinamento_limpo
lista1=[]
for tweet in n_rel1:
    for palavra in tweet.split(','):
        lista1.append(palavra)
serie_n_rel=pd.Series(lista1).value_counts()
tot_n_rel=serie_n_rel.sum() #total de palavras na respectiva série






In [21]:
#value_counts-refere-se ao numero de vezes da palavra na série dos tweets relevantes positivos
rel_pos=train[train.Relevancia=='Relevante positiva']
rel_pos1=rel_pos.Treinamento_limpo
lista2=[]
for tweet in rel_pos1:
    for palavra in tweet.split(','):
        lista2.append(palavra)
serie_pos=pd.Series(lista2).value_counts()
tot_pos=serie_pos.sum() #total de palavras na respectiva série



In [22]:
#value_counts-refere-se ao numero de vezes da palavra na série dos tweets relevantes negativos
rel_neg=train[train.Relevancia=='Relevante negativa']
rel_neg1=rel_neg.Treinamento_limpo
lista3=[]
for tweet in rel_neg1:
    for palavra in tweet.split(','):
        lista3.append(palavra)
serie_neg=pd.Series(lista3).value_counts()
tot_neg=serie_neg.sum() #total de palavras na respectiva série






In [27]:
#listas 4 e 5 -referem-se respectivamente as palavras(index) e a contagem de cada palavra(values) na respectiva série
lista4=[list(serie_n_rel.index),list(serie_pos.index),list(serie_neg.index)]
lista5=[list(serie_n_rel.values),list(serie_pos.values),list(serie_neg.values)]
#guarda as contagens de cada palavra em uma lista
contagem=[]
for i in range(len(lista4)):
    dicio = {}
    for j in range(len(lista4[i])):
        dicio[lista4[i][j]]=lista5[i][j]
    contagem.append(pd.Series(dicio))



In [28]:
#retorna o total de palavras não repetidas em relacao as três classificações
tol = (serie_n_rel + serie_pos + serie_neg).index.nunique()
tol

2169

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

# 3°etapa-LImpando a base de dados do teste

In [55]:
def limpando(tweet):
    return remove_stopwords(remove_outros(cleanup_tweet(str(tweet))))


test_tweets = []
for tweet in test['Teste']:
    test_tweets.append(limpando(tweet))

# Cria coluna de pré processados nos dados de teste
series_test = pd.Series(test_tweets)
test['Pré Processado'] = series_test

#substitui a relevancia de numeros para categorias
test['Relevancia'] = test['Relevancia'].astype('category')

test['Relevancia'].cat.categories = ['Relevante negativa', 'Não relevante', 'Relevante positiva']
test.head(6)

,Teste,Relevancia,Pré Processado,Naive Bayes
0,@elissanmanoel tão costurando brahma ? hahaha ...,Não relevante,"[elissanmanoel, tão, costurando, brahma, hahah...",Relevante positiva
1,rt @nayaralucas11: era só uma praia e brahma t...,Não relevante,"[nayaralucas11, praia, brahma, trincando]",Relevante positiva
2,arrumando meu guarda roupa eis que encontro um...,Não relevante,"[arrumando, guarda, roupa, eis, encontro, ling...",Não Relevante
3,desceu foi whisky misturado c brahma kkkkkkkkk...,Não relevante,"[desceu, whisky, misturado, c, brahma, kkkkkkk...",Relevante positiva
4,rt @carolaguiarcm: hj eu só queria estar numa ...,Não relevante,"[carolaguiarcm, hj, queria, estar, mesa, skol,...",Relevante positiva
5,vó beber brahma fodaze,Não relevante,"[vó, beber, brahma, fodaze]",Relevante positiva


# Colocando as probabilidades de cada classe de relevancia

In [56]:
tabela=train.Relevancia.value_counts()
soma=train.Relevancia.value_counts().sum()
n_rel=tabela[0]/soma
rel_pos=tabela[1]/soma
rel_neg=tabela[2]/soma




# 4°etapa:Aplica a suavização de laplace 

In [57]:
def laplace(tweet):
   
    prob_nao_relevante=1
    prob_relevante_positivo=1
    prob_relevante_negativo=1
    #suavização de laplace para a 1° série
    for palavra in tweet:
        if (palavra not in contagem[0].index):
            prob_nao_relevante *=(1*n_rel)/(tot_n_rel+tol)
        else: 
            prob_nao_relevante *= (contagem[0][palavra] + 1)*(n_rel)/(tot_n_rel+tol)
        
    #suavização de laplace para a 2° série
    for palavra in tweet:
        if (palavra not in contagem[1].index):
            prob_relevante_positivo *=(1*rel_pos)/(tot_pos+tol)
        else:
            prob_relevante_positivo *= (contagem[1][palavra] + 1)/(tot_pos+tol)
            
     #suavização de laplace para a 3° série
    for palavra in tweet:
        if (palavra not in contagem[2].index):
            prob_relevante_negativo *=(1*rel_neg)/(tot_neg+tol)
        else:
            prob_relevante_negativo *= (contagem[2][palavra] + 1)*(rel_neg)/(tot_neg+tol)
            

    #agrega as probabilidades de cada frase em uma lista e acha o máximo em relação aos 3 valores
    list_prob = [prob_nao_relevante,prob_relevante_positivo,prob_relevante_negativo]
    index = list_prob.index(max(list_prob))
    
    #retorna onde a frase se encaixa(Naive Bayes)       
    if index == 0:
        return 'Não Relevante'
    elif index==1:
        return 'Relevante positiva'
    elif index==2:
        return 'Relevante negativa'


In [58]:
# Para cada tweet de teste, aplica o modelo e compara com o rótulo dado
classificacao_modelo = []
for i in range(0, len(test['Teste'])):
    classificacao_modelo.append(laplace(test['Pré Processado'][i]))
    
    

# Cria coluna nos dados de teste com os resultados
test["Naive Bayes"] = classificacao_modelo

# Exibe comparativo dos resultados do modelo com os rótulos (em porcentagem)
ct = pd.crosstab(test['Relevancia'], test['Naive Bayes'], normalize = True).round(4)*100
ct



Naive Bayes,Não Relevante,Relevante positiva
Relevancia,,
Relevante negativa,0.31,2.45
Não relevante,6.73,44.34
Relevante positiva,2.75,43.43


In [59]:
serie_0_test = test[test["Relevancia"] == 'Não relevante']
serie_1_test = test[test["Relevancia"] == 'Relevante positiva']
serie_2_test = test[test['Relevancia'] == 'Relevante negativa']

# Valores iniciais das quantidades de cada caso
verdadeiros_naorelevantes = 0
falsos_naorelevantes = 0
verdadeiros_positivos = 0
falsos_positivos = 0
verdadeiros_negativos = 0
falsos_negativos = 0

# Calcula a quantidade de tweets de cada um dos casos acima
for i in range(len(test['Relevancia'])):
    if test['Relevancia'][i] == 'Não relevante' and test['Naive Bayes'][i] == 'Não relevante':
        verdadeiros_naorelevantes += 1
    elif test['Relevancia'][i] == 'Não relevante' and test['Naive Bayes'][i] != 'Não relevante':
        falsos_naorelevantes += 1
    elif test['Relevancia'][i] == 'Relevante positiva' and test['Naive Bayes'][i] == 'Relevante positiva':
        verdadeiros_positivos += 1
    elif test['Relevancia'][i] == 'Relevante positiva' and test['Naive Bayes'][i] != 'Relevante positiva':
        falsos_positivos += 1
    elif test['Relevancia'][i] == 'Relevante negativa' and test['Naive Bayes'][i] == 'Relevante negativa':
        verdadeiros_negativos += 1
    elif test['Relevancia'][i] == 'Relevante negativa' and test['Naive Bayes'][i] != 'Relevante negativa':
        falsos_negativos += 1
    
    

# Retorna probabilidade para cada caso
print("Contagem\t\tProbabilidade (em %)")
print("----------------------------------------")
print("%s:\t%f" % ('Verdadeiros Não Relevantes', (verdadeiros_naorelevantes/len(serie_0_test['Relevancia'])*100)))
print("%s:\t\t%f" % ('Falsos Não Relevantes', (falsos_naorelevantes/len(serie_0_test['Relevancia'])*100)))
print("%s:\t\t%f" % ('Verdadeiros Positivos', (verdadeiros_positivos/len(serie_1_test['Relevancia'])*100)))
print("%s:\t\t%f" % ('Falsos Positivos', (falsos_positivos/len(serie_1_test['Relevancia'])*100)))
print("%s:\t\t%f" % ('Verdadeiros Negativos', (verdadeiros_negativos/len(serie_2_test['Relevancia'])*100)))
print("%s:\t\t%f" % ('Falsos Negativos', (falsos_negativos/len(serie_2_test['Relevancia'])*100)))

Contagem		Probabilidade (em %)
----------------------------------------
Verdadeiros Não Relevantes:	0.000000
Falsos Não Relevantes:		100.000000
Verdadeiros Positivos:		94.039735
Falsos Positivos:		5.960265
Verdadeiros Negativos:		0.000000
Falsos Negativos:		100.000000


___
### Concluindo

De acordo com os resultados obtidos pelo classificador e pelo cálculo das probabilidades, chegamos a conclusão que o classificador desenvolvido a partir da planilha de Testes do Excel não é exato para a classificação dos tweets em questão de relevância. Isso se deve ao fato de os classificados como Relevantes positivos na planilha Treinamento excedem em grande quantidade as outras categorias. Tendo isso em vista, pensando nos tweets negativos, obtivemos um pequeno numero de amostras e, portanto, o classificador teve uma exatidão maior. Por outro lado, tendo em vista os tweets positivos, como o numero de amostras foi maior, o classificador teve uma exatidão menor. Além disso não foi observada grande uniformidade entre a coleta de dados referente ao treinamento e a coleta de dados referente ao teste. 

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Aperfeiçoamento:

### Como melhorar o modelo? 

Pode-se melhorar o desempenho do classificador com algumas práticas:

- **Aumentar o banco de dados de treinamento com mais tweets**<br>
Para um número maior de tweets obtem-se maior variedade de palavras e,dessa maneira,maior distribuição de probabilidades para cada palavra.Na parte de obtenção de dados o aumento do número de palavras buscadas no treinamento e teste pode ser uma solução para a melhora desse ponto.

- **Melhorar classificação dos tweets**<br>
Após a coleta de tweets o grupo obteve muitas palavras e após a análise de sentimentos em relação a cada tweet o número de tweets relevantes positivos foi em ,grande medida, dominante em relação as outras categorias e influenciou na precisão de falsos e verdadeiros obtidos posteriormente.Logo,melhorar a análise de sentimentos e fundamental para a soluçõa do ponto citado




___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**